<a href="https://colab.research.google.com/github/zzyy-gh/stonks_scrape/blob/main/stonks_scrape(checker).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# scrape stock data daily from finviz
# data: tkr, pb, sgqq, s, f, (sf), si, sti, fsh, de, opm, prfm, weekc, monc, quartc, halfc, yearc, weekv, monv, p, dayc, v
# run on UTC time 0000, 4 hours after NYC: use yesterday's date in utc to get latest market data
import requests
from datetime import datetime, timedelta
import time
import multiprocessing
import json
from bs4 import BeautifulSoup

# ---------- functions ----------

def cleanData(rows):
    
    def cleanText(dataName, data):
        try:
            cleanItem[dataName] = data
        except:
            pass
        return

    def cleanSimpleFloat(dataName, data):
        try:
            if data != '-':
                cleanItem[dataName] = data
        except:
            pass
        return
    
    def cleanCommaInt(dataName, data):
        try:
            if data != '-':
                cleanItem[dataName] = data.replace(',', '')
        except:
            pass
        return
    
    def cleanPercFloat(dataName, data):
        try:
            if data != '-':
                cleanItem[dataName] = data[:-1]
        except:
            pass
        return
    
    def cleanMBFloat(dataName, data):
        try:
            if data != '-':
                if data[-1] == 'M':
                    cleanItem[dataName] = str(float(data[:-1]) * 1000000)
                elif data[-1] == 'B':
                    cleanItem[dataName] = str(float(data[:-1]) * 1000000000)              
        except:
            pass
        return
    
    date = (datetime.now() - timedelta(1)).strftime('%Y%m%d')
    pageCleanList = []
    rawList = []
    
    # extract raw data
    for row in rows:
        td = row.find_all('td')
        rawList.append([x.text for x in td])
        
    # save cleaned data
    for item in rawList:
        cleanItem = {}
        cleanText('tkr', item[0])
        cleanSimpleFloat('pb', item[1])
        cleanPercFloat('sgqq', item[2])
        cleanMBFloat('s', item[3])
        cleanMBFloat('f', item[4])
        cleanPercFloat('si', item[5])
        cleanPercFloat('sti', item[6])
        cleanPercFloat('fsh', item[7])
        cleanSimpleFloat('de', item[8])
        cleanPercFloat('opm', item[9])
        cleanPercFloat('prfm', item[10])
        cleanPercFloat('weekc', item[11])
        cleanPercFloat('monc', item[12])
        cleanPercFloat('quartc', item[13])
        cleanPercFloat('halfc', item[14])
        cleanPercFloat('yearc', item[15])
        cleanPercFloat('weekv', item[16])
        cleanPercFloat('monv', item[17])
        cleanSimpleFloat('p', item[18])
        cleanPercFloat('dayc', item[19])
        cleanCommaInt('v', item[20])
        try:
            if cleanItem['s'] and cleanItem['f']:
                cleanItem['sf'] = str(round(item[3]/item[4], 2))
        except:
            pass
        cleanItem['uuid'] = date + item[0]
        cleanItem['date'] = date

        pageCleanList.append(cleanItem)
    
    return pageCleanList

def scrape_n_store1(stopPage, finvizUrl1, finvizUrl2, faker, rowIncrement):

    # data
    row = 1
    cleanList = []

    # iterate through all the webpages to obtain and clean data
    start = time.time()
    while True:

        # fetch webpage
        try:
            url = finvizUrl1 + str(row) + finvizUrl2
            page = requests.get(url, headers=faker)
        except:
            print('Failed to retrieve row ' + str(row) + '. (1)')
            break

        # get clean data
        nicePage = BeautifulSoup(page.content, 'html.parser')
        rows = nicePage.find_all('tr', class_='table-dark-row-cp') + nicePage.find_all('tr', class_='table-light-row-cp')
        # if ads pops up, refresh
        if len(rows) == 0:
            # fetch webpage
            try:
                print('Retrieving refreshed page.')
                page = requests.get(url, headers=faker)
            except:
                print('Failed to retrieve refreshed page.')
                return
            nicePage = BeautifulSoup(page.content, 'html.parser')
            rows = nicePage.find_all('tr', class_='table-light-row-cp') + nicePage.find_all('tr', class_='table-dark-row-cp')
        cleanList.extend(cleanData(rows))

        # exit loop if on the last page, else continue to next page
        stopper = nicePage.find_all('a', class_='tab-link', string=stopPage)
        nextBtn = nicePage.find_all('a', class_='tab-link',string='next')
        if len(stopper) > 0 or len(nextBtn) == 0:
            print('Data scraping has ended. The first row number of the last page is ' + str(row) + '. (1)')
            break
        else:
            row += rowIncrement
            continue   
    print(str(time.time() - start) + ' seconds have elapsed for scraping. (1)')

    # return function if nothing is scraped
    if len(cleanList) == 0:
        print('No data is scraped. (1)')
        return
    
    print(len(cleanList))
    print(cleanList)
    return
    
def scrape_n_store2(startIndex, finvizUrl1, finvizUrl2, faker, rowIncrement):

    # data
    row = startIndex
    cleanList = []
                
    # iterate through all the webpages to obtain and clean data
    start = time.time()
    while True:

        # fetch webpage
        try:
            url = finvizUrl1 + str(row) + finvizUrl2
            page = requests.get(url, headers=faker)
        except:
            print('Failed to retrieve row ' + str(row) + '. (2)')
            break

        # get clean data
        nicePage = BeautifulSoup(page.content, 'html.parser')
        rows = nicePage.find_all('tr', class_='table-dark-row-cp') + nicePage.find_all('tr', class_='table-light-row-cp')
        # if ads pops up, refresh
        if len(rows) == 0:
            # fetch webpage
            try:
                print('Retrieving refreshed page.')
                page = requests.get(url, headers=faker)
            except:
                print('Failed to retrieve refreshed page.')
                return
            nicePage = BeautifulSoup(page.content, 'html.parser')
            rows = nicePage.find_all('tr', class_='table-light-row-cp') + nicePage.find_all('tr', class_='table-dark-row-cp')
        cleanList.extend(cleanData(rows))

        # exit loop if on the last page, else continue to next page
        nextBtn = nicePage.find_all('a', class_='tab-link',string='next')
        if len(nextBtn) == 0:
            print('Data scraping has ended. The first row number of the last page is ' + str(row) + '. (2)')
            break
        else:
            row += rowIncrement
            continue   
    print(str(time.time() - start) + ' seconds have elapsed for scraping. (2)')

    # return function if nothing is scraped
    if len(cleanList) == 0:
        print('No data is scraped. (2)')
        return
    
    print(len(cleanList))
    print(cleanList)
    return
    
# ---------- main function ----------

def lambda_handler(event, context):
    
    # ---------- global variables ----------

    marketIsOpen = False
    yesterday = datetime.now() - timedelta(1)
    date = yesterday.strftime('%Y-%m-%d')
    month = yesterday.strftime('%m')
    year = yesterday.strftime('%Y')
    calendarApi = 'https://sandbox.tradier.com/v1/markets/calendar'
    calendarApiToken = 't0XrEyArrcq6EJZAAbP6zbZDl9FA'
    finvizUrl1 = 'https://finviz.com/screener.ashx?v=152&o=ticker&r='
    finvizUrl2 = '&c=1,11,23,24,25,26,28,30,38,40,41,42,43,44,45,46,50,51,65,66,67'
    faker = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    rowIncrement = 20
    startIndex = 4101
    stopPage = '205'

    # ---------- check if market is open ----------

    calendar = requests.get(calendarApi, 
                     headers={'Authorization': calendarApiToken, 'Accept': 'application/json'},
                     params={'month': month, 'year': year})

    dayListInThisMonth = calendar.json()['calendar']['days']['day']
    try:
        for day in dayListInThisMonth:
            if day['date'] == date:
                if day['status'] == 'open':
                    marketIsOpen = True
                    break
        if marketIsOpen:
            print('Market is open.')
        else:
            print('Market is closed.')        
    except:
        print('Failed to check market\'s status.')

    # ---------- return function if market is closed ----------

    if not marketIsOpen:
        return

    # ---------- scrape data from finviz ----------

    scrape_n_store1(stopPage, finvizUrl1, finvizUrl2, faker, rowIncrement)
    scrape_n_store2(startIndex, finvizUrl1, finvizUrl2, faker, rowIncrement)

    return 


In [ ]:
lambda_handler(1, 2)

Market is open.
